# SQL query from table names - Continued

In [1]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables from the .env file
_ = load_dotenv(find_dotenv())

# Get the OpenAI API key from environment variables
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Set the OpenAI API key in the OpenAI Python package
openai.api_key = OPENAI_API_KEY

## The old Prompt

In [4]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [5]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [6]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [7]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [11]:
import openai
import os
from dotenv import load_dotenv

# Load your .env file (optional, if you're using an environment file to store keys)
load_dotenv()

# Set the OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')  # Or directly input the key: 'your-api-key-here'

# Sample function to use OpenAI
def get_sql_response(user_message):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a SQL assistant."},
            {"role": "user", "content": user_message}
        ],
        temperature=0.5,
    )
    return response['choices'][0]['message']['content']




In [22]:
import openai

def return_CCRMSQL(user_message, context):
    newcontext = context.copy()
    newcontext.append({'role': 'user', 'content': "question: " + user_message})
    
    # Generate the response
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=newcontext,
        temperature=0.5
    )
    
    return response['choices'][0]['message']['content']

# Example usage
old_context_user = old_context.copy()
print(return_CCRMSQL("""
you are a bot to assist in create SQL commands, all your answers should start with 'This is your SQL,' and after that an SQL that can do what the user requested. Your Database is composed of a SQL database with some tables. Try to maintain the SQL order simple. Put the SQL command in white letters with a black background, and just after a simple and concise text explaining how it works. If the user asks for something that cannot be solved with SQL, just answer something nice and simple, maximum 10 words, asking for something that can be solved with SQL.
""", old_context_user))



Something that can be solved with SQL, please.


In [23]:
#new
print(return_CCRMSQL("YOUR QUERY HERE", context_user))

I see that you haven't provided any tables or specific questions to answer. Please provide the tables and questions you would like me to use for the queries.


In [24]:
#old
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

This is your SQL: 

```sql
SELECT * FROM employees;
```

Explanation: This SQL query selects all data from the "employees" table.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [25]:
""" Prompt 1: Basic Setup
Start by defining the structure and the behavior of the bot. This includes setting up a database, creating SQL commands, and making sure the responses are in a simple format. """
# Basic Setup: Define the database and SQL command structure
def return_CCRMSQL(user_message, context):
    new_context = context.copy()
    new_context.append({'role': 'user', 'content': user_message})
    
    # Sample SQL database schema
    tables = {
        'employees': {'ID_usr': 'int', 'name': 'varchar'},
        'salary': {'ID_usr': 'int', 'year': 'date', 'salary': 'float'}
    }
    
    # Placeholder SQL query construction (to be replaced in later prompts)
    query = f"SELECT * FROM {list(tables.keys())[0]} WHERE ID_usr = 1;"
    
    return query  # Returning a simple query as a placeholder

# Initial prompt
old_context_user = [{"role": "system", "content": "You are an SQL assistant."}]
print(return_CCRMSQL("Show me all employees.", old_context_user))


SELECT * FROM employees WHERE ID_usr = 1;


In [26]:
""" Prompt 2: Expanding Functionality
Here, we enhance the bot's ability to generate more complex SQL queries based on user inputs, including JOIN operations and conditi """
# Expanding SQL query functionality
def return_CCRMSQL(user_message, context):
    new_context = context.copy()
    new_context.append({'role': 'user', 'content': user_message})

    # Extracting action from user_message
    if "employees with salary" in user_message:
        query = """
        SELECT employees.name, salary.salary
        FROM employees
        JOIN salary ON employees.ID_usr = salary.ID_usr
        WHERE salary.salary > 50000 AND salary.year = '2023';
        """
    else:
        query = "SELECT * FROM employees;"
    
    return query

# Prompt with more complexity
old_context_user = [{"role": "system", "content": "You are an SQL assistant."}]
print(return_CCRMSQL("Get employees with salary over 50000 in 2023.", old_context_user))




        SELECT employees.name, salary.salary
        FROM employees
        JOIN salary ON employees.ID_usr = salary.ID_usr
        WHERE salary.salary > 50000 AND salary.year = '2023';
        


In [27]:
""" Prompt 3: Adding Explanation and Error Handling
Now, we add explanations for the generated SQL and handle scenarios where the request cannot be solved with SQL. """
# Adding explanations and handling unsolvable SQL requests
def return_CCRMSQL(user_message, context):
    new_context = context.copy()
    new_context.append({'role': 'user', 'content': user_message})

    # Check for specific conditions in the message
    if "employees with salary" in user_message:
        query = """
        SELECT employees.name, salary.salary
        FROM employees
        JOIN salary ON employees.ID_usr = salary.ID_usr
        WHERE salary.salary > 50000 AND salary.year = '2023';
        """
        explanation = "This SQL query retrieves the names and salaries of employees who earned more than 50,000 in 2023."
    else:
        query = "This request cannot be solved with SQL."
        explanation = "Please ask something solvable with SQL, like querying data from tables."
    
    return query, explanation

# Final prompt with explanation
old_context_user = [{"role": "system", "content": "You are an SQL assistant."}]
query, explanation = return_CCRMSQL("Show all employees who received a bonus.", old_context_user)
print(query)
print(explanation)


This request cannot be solved with SQL.
Please ask something solvable with SQL, like querying data from tables.


Summary:
Prompt 1 defines a basic SQL structure.
Prompt 2 introduces more complex SQL generation, including conditions.
Prompt 3 adds explanations and error handling to guide the user.